In [ ]:
pip install tensorflow


In [3]:
import tensorflow as tf
import keras
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [ ]:
# Set seed for reproducibility
tf.random.set_seed(42)

test_normal_path = "test/NORMAL/*.jpeg"
test_pneumonia_path = "test/PNEUMONIA/*.jpeg"

train_normal_path = "train/NORMAL/*.jpeg"
train_pneumonia_path = "train/PNEUMONIA/*.jpeg"

val_normal_path = "val/NORMAL/*.jpeg"
val_pneumonia_path = "val/PNEUMONIA/*.jpeg"

last_test_normal_path = "last_test/*.jpeg"

# Loading test image paths
test_normal_files = glob.glob(test_normal_path)
test_pneumonia_files = glob.glob(test_pneumonia_path)

# Loading training image paths
train_normal_files = glob.glob(train_normal_path)
train_pneumonia_files = glob.glob(train_pneumonia_path)

# Loading validation image paths
val_normal_files = glob.glob(val_normal_path)
val_pneumonia_files = glob.glob(val_pneumonia_path)

# Loading last test image paths
last_test_files = glob.glob(last_test_normal_path)

print("Number of normal test images:", len(test_normal_files))
print("Number of pneumonia test images:", len(test_pneumonia_files))
print("Number of normal training images:", len(train_normal_files))
print("Number of pneumonia training images:", len(train_pneumonia_files))
# print("Number of normal validation images:", len(val_normal_files))
# print("Number of pneumonia validation images:", len(val_pneumonia_files))


In [6]:
# Creation of matrix and label arrays

# Train
train_matrix = []
train_label = []

# Test
test_matrix = []
test_label = []

# Validation
val_matrix = []
val_label = []

last_test_matrix = []
last_test_label = []

In [7]:
def image_to_matrix(files, label, matrix_array, label_array):
    for file in files:
        file_data = tf.io.read_file(file)
        # Decode the image into a pixel matrix
        matrix = tf.image.decode_jpeg(file_data, channels=3)

        # Resize the matrix
        matrix = tf.image.resize(matrix, [150, 150])
        
        # Normalize the pixels between 0 and 1
        matrix = matrix / 255.0
        
        # Add the matrix to the matrix array
        matrix_array.append(matrix)
        
        # Add the label to the label array
        # 0 = Normal
        # 1 = Pneumonia
        label_array.append(label)

In [ ]:
image_to_matrix(train_normal_files, 0, train_matrix, train_label)
image_to_matrix(train_pneumonia_files, 1, train_matrix, train_label)

image_to_matrix(test_normal_files, 0, test_matrix, test_label)
image_to_matrix(test_pneumonia_files, 1, test_matrix, test_label)

image_to_matrix(val_normal_files, 0, val_matrix, val_label)
image_to_matrix(val_pneumonia_files, 1, val_matrix, val_label)

image_to_matrix(last_test_files, 0, last_test_matrix, last_test_label)

print("Number of images in the train array:", len(train_matrix))
print("Number of labels in the train array:", len(train_label))

print("Number of images in the test array:", len(test_matrix))
print("Number of labels in the test array:", len(test_label))

print("Number of images in the validation array:", len(val_matrix))
print("Number of labels in the validation array:", len(val_label))


In [9]:
# Set seed for reproducibility
tf.random.set_seed(42)

# Create the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Summary du modèle
model.summary()

In [11]:
# Model Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Conversion of arrays to tensors
train_matrix_np = np.array(train_matrix, dtype=np.float32)
train_label_np = np.array(train_label, dtype=np.float32)
test_matrix_np = np.array(test_matrix, dtype=np.float32)
test_label_np = np.array(test_label, dtype=np.float32)
val_matrix_np = np.array(val_matrix, dtype=np.float32)
val_label_np = np.array(val_label, dtype=np.float32)
last_test_matrix_np = np.array(last_test_matrix, dtype=np.float32)

history = model.fit(train_matrix_np, train_label_np, epochs=5, 
                    validation_data=(test_matrix_np, test_label_np))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_matrix_np,  test_label_np, verbose=2)

In [ ]:
test_loss, test_acc = model.evaluate(test_matrix_np,  test_label_np, verbose=2)
print(test_loss, test_acc)

In [ ]:
print(test_acc)

In [16]:
# Saving the model
model.save('model.h5')